In [1]:
#dhaval sat on a sofa and ate a samosa
#bow
#Dhaval  sat  on   a  sofa  and  ate...

#bi-gram
#Dhaval sat   sat on   on a    a sofa   sofa and   and ate

#tri-gram
#Dhaval sat on       sat on a       on a sofa      a sofa and ......
#ans soon


In [2]:
#bag of n is a special case of bag of ngram where n is 1
#limitation of nggram model
#as n increased, dimensionality sparsity increases
#2.doensnt address out of vocabulary(oov) problem


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
v=CountVectorizer()
v.fit(["Thor Hathodwala is looking for a job"])
v.vocabulary_

{'thor': 5, 'hathodwala': 1, 'is': 2, 'looking': 4, 'for': 0, 'job': 3}

In [4]:
v=CountVectorizer(ngram_range=(2,2))
v.fit(["Thor Hathodwala is looking for a job"])
v.vocabulary_

{'thor hathodwala': 4,
 'hathodwala is': 1,
 'is looking': 2,
 'looking for': 3,
 'for job': 0}

In [5]:
v=CountVectorizer(ngram_range=(1,3))
v.fit(["Thor Hathodwala is looking for a job"])
v.vocabulary_
#it will generate single token, bigram and trigram

{'thor': 12,
 'hathodwala': 2,
 'is': 5,
 'looking': 9,
 'for': 0,
 'job': 8,
 'thor hathodwala': 13,
 'hathodwala is': 3,
 'is looking': 6,
 'looking for': 10,
 'for job': 1,
 'thor hathodwala is': 14,
 'hathodwala is looking': 4,
 'is looking for': 7,
 'looking for job': 11}

In [6]:
corpus=[
    "Thor ate pizza",
    "Loki is tail",
    "Loki is eating pizza"
]

In [7]:
import spacy
nlp=spacy.load("en_core_web_sm")
def preprocessing(text):
    doc=nlp(text)
    filtered_token=[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_token.append(token.lemma_)
    return " ".join(filtered_token)
preprocessing("Thore ate pizza.,")
    

'thore eat pizza'

In [8]:
corpus_processed=[preprocessing(text) for text in corpus]
corpus_processed

['Thor eat pizza', 'Loki tail', 'Loki eat pizza']

In [9]:
v=CountVectorizer(ngram_range=(1,2))
v.fit(corpus_processed)
v.vocabulary_

{'thor': 7,
 'eat': 0,
 'pizza': 5,
 'thor eat': 8,
 'eat pizza': 1,
 'loki': 2,
 'tail': 6,
 'loki tail': 4,
 'loki eat': 3}

In [10]:
v.transform(["thor eat pizza"]).toarray()

array([[1, 1, 0, 0, 0, 1, 0, 1, 1]], dtype=int64)

In [11]:
v.transform(['meet loves pizza']).toarray()

array([[0, 0, 0, 0, 0, 1, 0, 0, 0]], dtype=int64)

In [12]:
import pandas as pd
df=pd.read_json("news_dataset.json")
print(df.shape)
df.head()

(12695, 2)


,text,category
0,Watching Schrödinger's Cat Die University of C...,SCIENCE
1,WATCH: Freaky Vortex Opens Up In Flooded Lake,SCIENCE
2,Entrepreneurs Today Don't Need a Big Budget to...,BUSINESS
3,These Roads Could Recharge Your Electric Car A...,BUSINESS
4,Civilian 'Guard' Fires Gun While 'Protecting' ...,CRIME


In [13]:
df.category.value_counts()

BUSINESS    4254
SPORTS      4167
CRIME       2893
SCIENCE     1381
Name: category, dtype: int64

In [14]:
#undersampling
min_samples=1381
df_business=df[df.category=='BUSINESS'].sample(min_samples,random_state=37)
df_business

,text,category
753,Iowa’s 60 Million Laying Hens Aren’t Being Mon...,BUSINESS
4494,"Dollar Charges To 14-Year High, Bond Tantrum I...",BUSINESS
7432,GoDaddy Pulls The Plug On Neo-Nazi Website The...,BUSINESS
3766,"Women in Business Q&A: Darline Jean, COO, Puls...",BUSINESS
9189,Rushion McDonald: Architect Behind the Steve H...,BUSINESS
...,...,...
2470,Tim Cook Sends Memo To Reassure Apple Employee...,BUSINESS
5809,"Like a Bad Neighbor, Chevron Is There The fede...",BUSINESS
6226,Asian Americans versus Asian Americans Asian i...,BUSINESS
11899,Another Way Companies Make It Harder For New M...,BUSINESS


In [15]:
df_sports=df[df.category=='SPORTS'].sample(min_samples,random_state=37)
df_crime=df[df.category=='CRIME'].sample(min_samples,random_state=37)
df_science=df[df.category=='SCIENCE'].sample(min_samples,random_state=37)

In [16]:
df_balanced=pd.concat([df_business,df_crime,df_sports,df_science],axis=0)
df_balanced

,text,category
753,Iowa’s 60 Million Laying Hens Aren’t Being Mon...,BUSINESS
4494,"Dollar Charges To 14-Year High, Bond Tantrum I...",BUSINESS
7432,GoDaddy Pulls The Plug On Neo-Nazi Website The...,BUSINESS
3766,"Women in Business Q&A: Darline Jean, COO, Puls...",BUSINESS
9189,Rushion McDonald: Architect Behind the Steve H...,BUSINESS
...,...,...
3901,Privatization Has Been a Colossal Flop We have...,SCIENCE
8865,"Astronomers Discover Tiny, Shy Moon Hiding In ...",SCIENCE
2303,Sisterhood: Powerful on Mars! I met three impr...,SCIENCE
7720,SOLVED? Massive Magnetic Star Mystery,SCIENCE


In [17]:
#machine learning model cannot take string so we will convert text into no
target={'BUSINESS':0,'SPORTS':1,'CRIME':2,'SCIENCE':3}
df_balanced['category_num']=df_balanced.category.map(target)

In [18]:
df_balanced.category.value_counts()

BUSINESS    1381
CRIME       1381
SPORTS      1381
SCIENCE     1381
Name: category, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    df_balanced.text,
    df_balanced.category_num,
    test_size=0.2,
    random_state=37,
    stratify=df_balanced.category_num
    
)
#The stratify parameter in this syntax is used to ensure that the target variable (in this case, "category_num" column in the dataframe "df_balanced") is equally distributed in both the training and testing sets. This is particularly useful when dealing with imbalanced datasets, where the distribution of the target variable is uneven

In [20]:
y_train.value_counts()

2    1105
0    1105
1    1105
3    1104
Name: category_num, dtype: int64

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [22]:
model=Pipeline([
    ('vectorizer_bow',CountVectorizer()),
    ('Multi NB',MultinomialNB())
])

In [23]:
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [25]:
print(classification_report(y_test,y_pred))
#first argument is truth and second argument is test

              precision    recall  f1-score   support

           0       0.79      0.92      0.85       276
           1       0.91      0.82      0.86       276
           2       0.88      0.89      0.89       276
           3       0.90      0.82      0.86       277

    accuracy                           0.87      1105
   macro avg       0.87      0.87      0.87      1105
weighted avg       0.87      0.87      0.87      1105



In [31]:
X_test[5:10]

1466     The Surprising Benefit Of Going Through Hard T...
169      5 Reasons You Will Fail At Working From Home W...
5498     SpaceX Executive Quits To Fight Trump As A Gra...
11679    Elizabeth Warren Just Launched A New Attack On...
2259     Solar Jobs Fell For The First Time In 7 Years ...
Name: text, dtype: object

In [32]:
y_pred[5:10]

array([3, 0, 0, 0, 0], dtype=int64)

In [33]:
y_test[5:10]

1466     3
169      0
5498     0
11679    0
2259     0
Name: category_num, dtype: int64